Import SysMLv2 Model:

In [1]:
import syside
import os

MAIN_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_DIR = "omg_single_example/"
# MODEL_DIR = "omg_introduction/" # mid complexity example
# MODEL_DIR = "omg_simple_vehicle/" # high complexity example
MODEL_DIR = "project_electronics_basics/"
# MODEL_DIR = "project_clean/"

# Construct full directory path
full_dir = os.path.join(MAIN_DIR, MODEL_DIR)

# Find the first file ending with .sysml
for file in os.listdir(full_dir):
    if file.endswith(".sysml"):
        MODEL_FILE_PATH = os.path.join(full_dir, file)
        break  # stop after finding the first .sysml file

(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Check Metamodel structure:

In [2]:
for doc in model.user_docs:
    print(doc)
    with doc.lock() as locked:
        print(f"Model sexp:\n {syside.sexp(locked.root_node)}")

Model sexp:
 (Namespace
  (OwningMembership
    (Package model
      (NamespaceImport)
      (OwningMembership
        (Package circuit_definitions
          (OwningMembership
            (PartDefinition ElectronicComponent
              (Subclassification)))
          (OwningMembership
            (PartDefinition Battery
              (Subclassification)
              (FeatureMembership
                (PortUsage +
                  (Subsetting)))
              (FeatureMembership
                (PortUsage -
                  (Subsetting)))
              (FeatureMembership
                (AttributeUsage voltage
                  (Subsetting)))))
          (OwningMembership
            (PartDefinition Resistor
              (Subclassification)
              (FeatureMembership
                (PortUsage a
                  (Subsetting)))
              (FeatureMembership
                (PortUsage b
                  (Subsetting)))
              (FeatureMembership
                (Attri

Check PartDefinition:

In [ ]:
for PartDefinition in model.nodes(syside.PartDefinition):
    print(PartDefinition)
    print(f"\tType::is_abstract: {PartDefinition.is_abstract}")
    print(f"\tOcurrenceDefinition::is_individual: {PartDefinition.is_individual}")
    print(f"\tDefinition::owned_allocations: {len(list(PartDefinition.owned_allocations))}")
    print(f"\tDefinition::owned_parts: {len(list(PartDefinition.owned_parts))}")

Check PartUsage:

In [ ]:
for PartUsage in model.nodes(syside.PartUsage):
    print(PartUsage)
    print(f"\tPartUsage::part_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tItemUsage::item_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tOccurrenceUsage::individual_definition: {PartUsage.individual_definition}")
    print(f"\tElement::declared_name: {PartUsage.declared_name}")
    print(f"\tElement::documentation: {len(list(PartUsage.documentation))}")
    print(f"\tElement::element_id: {PartUsage.element_id}")
    print(f"\tElement::owner: {PartUsage.owner}")
    print(f"\tElement::owning_membership: {PartUsage.owning_membership}")
    print(f"\tElement::owning_namespace: {PartUsage.owning_namespace}")
None

Export to JSON (minimal):

In [3]:
from pathlib import Path

assert len(model.user_docs) == 1

with model.user_docs[0].lock() as locked:
    json_output = syside.json.dumps(
        locked.root_node, syside.SerializationOptions.minimal()
    )

# Use current working directory (instead of __file__)
output_path = Path.cwd() / "export.json"

# Write JSON file
output_path.write_text(json_output, encoding="utf-8")

print(f"JSON saved to: {output_path}")


JSON saved to: /Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/export.json


Export to JSON (with options)

In [ ]:
from pathlib import Path

assert len(model.user_docs) == 1

options = syside.SerializationOptions()  # create an instance
options = options.with_options(
    use_standard_names=True,
    include_derived=True,
    include_redefined=True,
    include_default=True,
    include_optional=True,
    include_implied=True
)

with model.user_docs[0].lock() as locked:
    json_output = syside.json.dumps(
        locked.root_node, 
        options,
    )

# Use current working directory (instead of __file__)
output_path = Path.cwd() / "export.json"

# Write JSON file
output_path.write_text(json_output, encoding="utf-8")

print(f"JSON saved to: {output_path}")


Create cypher script for neo4j:

In [4]:
import json

with open("export.json") as f:
    data = json.load(f)

cypher = []

# 1. Create all nodes
for e in data:
    props = []
    for key in ("@id", "@type", "declaredName", "memberName", "operator", "value"):
        val = e.get(key)
        if val:
            field = "id" if key == "@id" else "type" if key == "@type" else key
            props.append(f'{field}:"{val}"')
    # cypher.append(f'CREATE (:`Node` {{{", ".join(props)}}});') # original
    # cypher.append(f'CREATE ({e.get("@type")}:Node {{{", ".join(props)}}});') # using @type as node label name
    cypher.append(f'CREATE (:{e.get("@type")} {{{", ".join(props)}}});') # using @type as node label

# 2. Create relationships
rels = {
    "owningRelatedElement": "OWNING_RELATED_ELEMENT",
    "ownedRelationship": "OWNED_RELATIONSHIP",
    "memberElement": "MEMBER_ELEMENT",
    "ownedRelatedElement": "OWNED_RELATED_ELEMENT",
    "owningRelationship": "OWNING_RELATIONSHIP",
    "general": "GENERAL",
    "specific": "SPECIFIC",
    "type": "TYPE",
    "typedFeature": "TYPED_FEATURE",
    "importedMembership": "IMPORTED_MEMBERSHIP",
    "memberName": "MEMBER_NAME",
    "memberShortName": "MEMBER_SHORT_NAME",
    "conjugatedType": "CONJUGATED_TYPE",
    "originalType": "ORIGINAL_TYPE"
}
for e in data:
    src_id = e["@id"]
    for k, rel in rels.items():
        if k in e:
            targets = e[k] if isinstance(e[k], list) else [e[k]]
            for t in targets:
                
                # ensure the target is a dict with an @id
                if not isinstance(t, dict) or "@id" not in t:
                    # skip plain strings, numbers, etc.
                    print("t doesn't contain @id objects")
                    continue
                tid = t.get("@id")

                if tid:
                    td_index = next((i for i, item in enumerate(data) if item["@id"] == t.get("@id")), None) # target data index
                    if td_index is None:
                        print(f"td_index is None! {e["@id"]} {e.get("@type")}")
                    else:    
                        cypher.append(
                            # f'MATCH (a:Node {{id:"{src_id}"}}), (b:Node {{id:"{tid}"}}) ' # original
                            f'MATCH (a:{e.get("@type")} {{id:"{src_id}"}}), (b:{data[td_index]['@type']} {{id:"{tid}"}}) ' #
                            f'CREATE (a)-[:{rel}]->(b);'
                        )

with open("export.cypher", "w") as f:
    f.write("\n".join(cypher))


t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't 